In [85]:
import pandas as pd
import gspread
import os
from datetime import datetime
import requests

In [86]:
# Baixa e lê a planilha de projetos em curso
r = requests.get("https://geoex.com.br/Sistema/Job/Download?data=eyJKb2JJZCI6IjI3NGY2ZGRhLWEzZGItNGNkMi05NmQ3LWJjNGE5NjlhYTFlNyJ9", headers={'cookie':'_ga=GA1.1.1408546827.1679344756; TemaEscuro=true; Home.Buscar.Texto=; ASP.NET_SessionId=r4jjibpbd1byxmnxyobgib30; ConsultarNota.Numero=9101900448; ConsultarProjeto.Numero=B-0808522; .ASPXAUTH=9160AE158E39CD37666221CCC1EE172AA3D6C30D63C299E69025FAA248B8FEEE872910F8217D345BEBAFA8AD129E9CE0501D9F4092229C8EB83E3AEDE5CF07C79105BB052DAA6F88D2B248C2B15A8AC2A2F797281B8BEA7D58C52744F8AE0996B6B6CC4A02D60C9D4E387A97521E74B64CBD9D7473E2844E175A55653C8307E6F35C2B584C4EAEF6D3F1A3A273D6779B1EFDCF6B9AD5C490580F616E06896FE4B2BDC327FC24234A36003E18BA1063F8E8CD2199A2FEF65164280952C81AEE5B3634C2735A896965D31272EC9869B5D9; _ga_ZBQMHFHTL8=GS1.1.1681316552.74.0.1681316552.0.0.0'})
open('projetos em curso.zip', 'wb').write(r.content)
projetos_em_curso = pd.read_csv('projetos em curso.zip', delimiter=';', encoding='ISO-8859-1', low_memory=False)
projetos_em_curso['Projeto'] = projetos_em_curso['ProjetoText'].str.extract(r'(\d\d\d\d\d\d\d)')
projetos_em_curso['Projeto'] = projetos_em_curso['Projeto'].fillna(0)
projetos_em_curso['Projeto'] = projetos_em_curso['Projeto'].astype(int)

In [87]:
# Inicia serviço do Google Sheets
gs = gspread.service_account(filename='service_account.json')

In [88]:
orcamentos = gs.open_by_key('14d0b2l65tMKZll6SAlqXqRqAjIEIHwkRhJI4YVgIi88')
materiais_orcados = orcamentos.worksheet('MATERIAIS').get_all_records(value_render_option="UNFORMATTED_VALUE")
materiais_orcados = pd.DataFrame(materiais_orcados)
materiais_orcados = materiais_orcados.loc[materiais_orcados['Projeto'] != '']   
materiais_orcados = materiais_orcados.rename(columns = {'Código': 'Material'})

In [89]:
carteira = gs.open_by_key('1yZUYH8x-LzlA7OqrExA7igqin1PG8h3TWQPt0UUJqxI')
carteira = carteira.worksheet('CARTEIRA').get_all_records()
carteira = pd.DataFrame(carteira)
carteira = carteira.loc[carteira['PROJETO'] != '']
carteira = carteira.rename(columns={'PROJETO':'Projeto'})
projeto_vs_unidade = carteira.groupby(['Projeto', 'UNIDADE'], as_index=False).count()[['Projeto', 'UNIDADE']]

In [90]:
programacao = gs.open_by_key('1ztV6DYCUkhefULyxJTBiaAKLB_x5zAHgK9icFqBDvf4')
programacao = programacao.worksheet('PROGRAMAÇÃO').get_all_records()
programacao = pd.DataFrame(programacao)
programacao = programacao.loc[programacao['Projeto'] != '']
programacao['Data Execução'] = pd.to_datetime(programacao['Data Execução'], format='%d/%m/%Y')

In [91]:
try:
    os.rename('cji3.xls', 'cji3.csv')
except FileExistsError:
    os.remove('cji3.csv')
    os.rename('cji3.xls', 'cji3.csv')
except FileNotFoundError:
    pass

cji3 = pd.read_csv('cji3.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
cji3 = cji3.loc[cji3['Unnamed: 1'] != '*']
cji3['Material'] = cji3['Material'].fillna(0)
cji3['Material'] = cji3['Material'].astype(int)
cji3 = cji3.loc[cji3['Material'] != 0]
cji3 = cji3.loc[cji3['Usuário'].str.contains('STC|SIR')]

cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace('.','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(',','.')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(' ','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].astype(float)

cji3['Def.proj.'] = cji3['Def.proj.'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)

In [92]:
try:
    os.rename('zmm370.xls', 'zmm370.csv')
except FileExistsError:
    os.remove('zmm370.csv')
    os.rename('zmm370.xls', 'zmm370.csv')
except FileNotFoundError:
    pass

zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
zmm370 = zmm370.loc[zmm370['Registro Final'] != 'X']
zmm370 = zmm370.loc[(zmm370['Tipo movimento'] == 221) | (zmm370['Tipo movimento'] == 921)]
zmm370 = zmm370.loc[zmm370['Recebedor'].str.contains('3ST')]

zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace('.','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(',','.')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(' ','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].astype(float)

zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace('.','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(',','.')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(' ','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].astype(float)

zmm370['Projeto'] = zmm370['Projeto'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)
zmm370 = zmm370.rename(columns={'Qtd Pendente': 'Qtd. Reservada'})

C:\Users\Hugo Viana\AppData\Local\Temp\ipykernel_8560\1020458792.py:9: DtypeWarning: Columns (0,4,6,7,11,12,16,18,20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)


In [93]:
reservas_criadas = zmm370[['Projeto', 'Material', 'Qtd. Reservada']].groupby(['Projeto', 'Material'], as_index=False).sum()
materiais_movimentados = cji3[['Def.proj.','Material', '  Qtd.entr.']].groupby(['Def.proj.', 'Material'], as_index=False).sum()
materiais_movimentados = materiais_movimentados.rename(columns={'Def.proj.': 'Projeto', '  Qtd.entr.': 'Qtd. Movimentada'})

In [94]:
data_inicio = datetime.strptime('2023-04-17', '%Y-%m-%d')
data_fim = datetime.strptime('2023-04-22', '%Y-%m-%d')

programacao_semana = programacao.loc[(programacao['Data Execução']>=data_inicio) & (programacao['Data Execução']<=data_fim)]
obras_programadas = programacao_semana[['Projeto']].groupby('Projeto').sum()

material_obras_programadas = obras_programadas.merge(materiais_orcados, on='Projeto', how='left').groupby(['Projeto', 'Material'], as_index=False).sum()
material_obras_programadas = material_obras_programadas.rename(columns={'Quantidade': 'Qtd. Orçada'})
material_obras_programadas = material_obras_programadas.merge(materiais_movimentados, on=['Projeto', 'Material'], how='left')
material_obras_programadas = material_obras_programadas.merge(reservas_criadas, on=['Projeto', 'Material'], how='left')
material_obras_programadas = material_obras_programadas.fillna(0)
material_obras_programadas['Material pendente de saque'] = material_obras_programadas['Qtd. Orçada'] - material_obras_programadas['Qtd. Movimentada']
material_obras_programadas = material_obras_programadas.loc[material_obras_programadas['Material pendente de saque'] > 0]
material_obras_programadas['Reserva pendente'] = material_obras_programadas['Material pendente de saque'] - material_obras_programadas['Qtd. Reservada']
material_obras_programadas = material_obras_programadas.loc[material_obras_programadas['Reserva pendente'] > 0]
material_obras_programadas = material_obras_programadas.merge(projeto_vs_unidade[['Projeto', 'UNIDADE']], on='Projeto')
material_obras_programadas = material_obras_programadas.merge(projetos_em_curso[['Projeto', 'StatusUsuarioENPUData']], on='Projeto', how='left')

In [97]:
material_obras_programadas.to_excel('Resultado.xlsx', index=False)

In [119]:
materiais_orcados.loc[materiais_orcados['Projeto'] == 265746].sort_values(by=['Material'])

,Projeto,Material,Descrição,Quantidade
25605,265746,211304,"TRAFO 1FF 10KVA 34,5KV 220V 3T MI",2
25639,265746,401010,"PARA-RAIOS RD 33,00KV 10KA",6
25604,265746,531005,"CH FUS DIST C 34,5KV 300A 3,5KA",7
25637,265746,536778,"ELO FUSIVEL 15,0/36,2KV 0,5H PRATA (FP)",5
25638,265746,536784,"ELO FUSIVEL 15,0/36,2KV 6K PRATA (FP)",2
...,...,...,...,...
25528,265746,3493315,ARRUELA LIS QUAD SAE1020 M18,208
25606,265746,3493325,"ARRUELA QUAD ACO 38 F14,00",112
25619,265746,4404025,ARAME ACO ZINCADO 8 BWG,45
25524,265746,4404026,ARAME ACO ZINCADO 12 BWG,1.5


In [116]:
materiais_movimentados.loc[materiais_movimentados['Projeto'] == 265746]

,Projeto,Material,Qtd. Movimentada
0,265746,211304,1.0
1,265746,401010,6.0
2,265746,531005,7.0
3,265746,536778,5.0
4,265746,536784,2.0
...,...,...,...
65,265746,3490080,92.0
66,265746,3493315,208.0
67,265746,3493325,112.0
68,265746,4404025,45.0


In [120]:
reservas_criadas.loc[reservas_criadas['Projeto'] == 265746].sort_values(by=['Material'])

,Projeto,Material,Qtd. Reservada
545,265746,211304,1.0
546,265746,2205000,16.1
547,265746,2223410,6.0
548,265746,2227021,200.0
549,265746,2230076,776.0
550,265746,2315000,60.0
551,265746,2411149,14.0
552,265746,3300012,2.0
553,265746,3300016,1.0
554,265746,3300152,3.0
